# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [87]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [88]:
weather_data_df = pd.read_csv("../output_data/weather_data.csv")
weather_data_df


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Faanui,PF,-16.4833,-151.7500,80.22,75,15,16.75,1651332108
1,Ulagan,RU,50.6321,87.9563,49.87,53,100,3.20,1651332108
2,Ubari,LY,26.5921,12.7805,99.66,5,0,15.35,1651332109
3,Hilo,US,19.7297,-155.0900,73.27,97,75,4.61,1651331924
4,Talnakh,RU,69.4865,88.3972,13.95,95,90,6.89,1651332109
...,...,...,...,...,...,...,...,...,...
552,Pochutla,MX,15.7432,-96.4661,85.82,50,59,3.44,1651332344
553,Iseyin,NG,7.9667,3.6000,94.51,37,59,5.48,1651332344
554,Gladstone,AU,-23.8500,151.2500,69.78,88,60,4.61,1651332344
555,Sarandi,BR,-23.4436,-51.8739,78.35,61,0,8.05,1651332345


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [89]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

In [90]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data_df[["Lat", "Lng"]]
humidity = weather_data_df["Humidity"].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

In [111]:
# Add Heatmap layer to map.
fig = gmaps.figure(map_type="HYBRID", center=(35.0, -80.0), zoom_level=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

    * A max temperature lower than 80 degrees but higher than 70.

    * Wind speed less than 10 mph.

    * Zero cloudiness.

    * Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

In [92]:
goldilocks_weather_df = weather_data_df.loc[(weather_data_df['Max Temp'] < 80) &
                        (weather_data_df['Max Temp'] > 70) &
                        (weather_data_df['Wind Speed'] < 10) &
                        (weather_data_df['Cloudiness'] == 0)]
goldilocks_weather_df
                        

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
36,Ponta do Sol,PT,32.6667,-17.1000,77.72,53,0,0.96,1651332121
63,Cabo San Lucas,MX,22.8909,-109.9124,77.50,45,0,2.30,1651332133
103,Crossville,US,35.9490,-85.0269,72.23,56,0,8.01,1651332150
225,Diego de Almagro,CL,-26.3667,-70.0500,72.90,28,0,4.88,1651332199
292,Santiago,CL,-33.4569,-70.6483,71.29,48,0,3.44,1651332103
303,Farafangana,MG,-22.8167,47.8333,72.88,84,0,4.16,1651332234
304,Zhezkazgan,KZ,47.8043,67.7144,71.58,37,0,4.47,1651332234
323,Springbok,ZA,-29.6643,17.8865,70.07,17,0,8.61,1651332242
337,Dolores Hidalgo Cuna de la Independencia Nacional,MX,21.1561,-100.9308,70.86,13,0,3.04,1651331990
468,Kalabo,ZM,-14.9700,22.6814,79.79,33,0,6.55,1651332309


### Hotel Map
* Store into variable named `hotel_df`.  
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [93]:
hotel_df = goldilocks_weather_df
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
36,Ponta do Sol,PT,32.6667,-17.1000,77.72,53,0,0.96,1651332121
63,Cabo San Lucas,MX,22.8909,-109.9124,77.50,45,0,2.30,1651332133
103,Crossville,US,35.9490,-85.0269,72.23,56,0,8.01,1651332150
225,Diego de Almagro,CL,-26.3667,-70.0500,72.90,28,0,4.88,1651332199
292,Santiago,CL,-33.4569,-70.6483,71.29,48,0,3.44,1651332103
303,Farafangana,MG,-22.8167,47.8333,72.88,84,0,4.16,1651332234
304,Zhezkazgan,KZ,47.8043,67.7144,71.58,37,0,4.47,1651332234
323,Springbok,ZA,-29.6643,17.8865,70.07,17,0,8.61,1651332242
337,Dolores Hidalgo Cuna de la Independencia Nacional,MX,21.1561,-100.9308,70.86,13,0,3.04,1651331990
468,Kalabo,ZM,-14.9700,22.6814,79.79,33,0,6.55,1651332309


In [94]:
hotel_df = goldilocks_weather_df
hotel_df["Hotel Name"] = ""

hotel_df

/var/folders/nk/670c2l4j02j1x3bk4kcm7qh80000gn/T/ipykernel_15613/222553520.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
36,Ponta do Sol,PT,32.6667,-17.1000,77.72,53,0,0.96,1651332121,
63,Cabo San Lucas,MX,22.8909,-109.9124,77.50,45,0,2.30,1651332133,
103,Crossville,US,35.9490,-85.0269,72.23,56,0,8.01,1651332150,
225,Diego de Almagro,CL,-26.3667,-70.0500,72.90,28,0,4.88,1651332199,
292,Santiago,CL,-33.4569,-70.6483,71.29,48,0,3.44,1651332103,
303,Farafangana,MG,-22.8167,47.8333,72.88,84,0,4.16,1651332234,
304,Zhezkazgan,KZ,47.8043,67.7144,71.58,37,0,4.47,1651332234,
323,Springbok,ZA,-29.6643,17.8865,70.07,17,0,8.61,1651332242,
337,Dolores Hidalgo Cuna de la Independencia Nacional,MX,21.1561,-100.9308,70.86,13,0,3.04,1651331990,
468,Kalabo,ZM,-14.9700,22.6814,79.79,33,0,6.55,1651332309,


In [95]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000, 
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {row['City']} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 36: Ponta do Sol.
Closest hotel in Ponta do Sol is Hotel do Campo.
------------
Retrieving Results for Index 63: Cabo San Lucas.


/Users/104rrc/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel in Cabo San Lucas is Hotel Tesoro Los Cabos.
------------
Retrieving Results for Index 103: Crossville.
Closest hotel in Crossville is Hampton Inn Crossville.
------------
Retrieving Results for Index 225: Diego de Almagro.
Closest hotel in Diego de Almagro is hotel luna del desierto.
------------
Retrieving Results for Index 292: Santiago.
Closest hotel in Santiago is Hotel Panamericano.
------------
Retrieving Results for Index 303: Farafangana.
Closest hotel in Farafangana is Miarina.
------------
Retrieving Results for Index 304: Zhezkazgan.
Closest hotel in Zhezkazgan is Hotel Zumrat.
------------
Retrieving Results for Index 323: Springbok.
Closest hotel in Springbok is Undulata Country Lodge.
------------
Retrieving Results for Index 337: Dolores Hidalgo Cuna de la Independencia Nacional.
Closest hotel in Dolores Hidalgo Cuna de la Independencia Nacional is Hotel Hidalgo.
------------
Retrieving Results for Index 468: Kalabo.
Closest hotel in Kalabo is Sikakubete G

In [96]:
column_names = ["Hotel Name", "City", "Country",  "Lat", "Lng", "Max Temp", 
                "Humidity", "Cloudiness", "Wind Speed"]
hotel_df = hotel_df.reindex(columns=column_names)
hotel_df.head(10)

,Hotel Name,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
36,Hotel do Campo,Ponta do Sol,PT,32.6667,-17.1000,77.72,53,0,0.96
63,Hotel Tesoro Los Cabos,Cabo San Lucas,MX,22.8909,-109.9124,77.50,45,0,2.30
103,Hampton Inn Crossville,Crossville,US,35.9490,-85.0269,72.23,56,0,8.01
225,hotel luna del desierto,Diego de Almagro,CL,-26.3667,-70.0500,72.90,28,0,4.88
292,Hotel Panamericano,Santiago,CL,-33.4569,-70.6483,71.29,48,0,3.44
303,Miarina,Farafangana,MG,-22.8167,47.8333,72.88,84,0,4.16
304,Hotel Zumrat,Zhezkazgan,KZ,47.8043,67.7144,71.58,37,0,4.47
323,Undulata Country Lodge,Springbok,ZA,-29.6643,17.8865,70.07,17,0,8.61
337,Hotel Hidalgo,Dolores Hidalgo Cuna de la Independencia Nacional,MX,21.1561,-100.9308,70.86,13,0,3.04
468,Sikakubete Guest House Kalabo Town Council,Kalabo,ZM,-14.9700,22.6814,79.79,33,0,6.55


In [97]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_two = hotel_df[["Lat", "Lng"]]

In [112]:
# Add marker layer ontop of heat map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

marker_layer = gmaps.marker_layer(locations_two, info_box_content=hotel_info)

# Display figure
fig = gmaps.figure()
fig = gmaps.figure(map_type="HYBRID", center=(10.0, -60.0), zoom_level=2)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))